In [41]:
import gxgp
from gxgp import DagGP, Node
import operator
import numpy as np
import matplotlib.pyplot as plt
from tqdm.auto import tqdm
import os
import re
import random
import copy

In [42]:
# -------------------------------
# Define custom mathematical functions
# -------------------------------
def custom_sin(x):
    """Sine function that handles numpy arrays."""
    return np.sin(x)

def custom_cos(x):
    """Cosine function that handles numpy arrays."""
    return np.cos(x)

def custom_exp(x):
    """Exponential function that handles numpy arrays safely."""
    # Clip x to prevent overflow
    clipped_x = np.clip(x, -100, 100)
    return np.exp(clipped_x)

def safe_div(x, y):
    """Division function that prevents division by zero."""
    return x / (y + 1e-6)  # Add small epsilon to avoid division by zero

def square(x):
    """Square function."""
    return x ** 2

def safe_exp(x):
    """Exponential function that prevents overflow."""
    # Clip x to a reasonable maximum value
    clipped_x = np.clip(x, -100, 100)  # Adjust these bounds as needed
    return np.exp(clipped_x)

def safe_log(x):
    """Protected logarithm: returns log(abs(x)+epsilon) to avoid log(0)."""
    return np.log(np.abs(x) + 1e-6)

def safe_sqrt(x):
    """Protected square root: returns sqrt(abs(x)) so the argument is non-negative."""
    return np.sqrt(np.abs(x))

def custom_tanh(x):
    """Hyperbolic tangent function that handles numpy arrays."""
    return np.tanh(x)

def safe_pow(x, y):
    """Protected power function: returns abs(x) raised to the power y safely.
       This version clips the base and avoids division by zero issues."""
    epsilon = 1e-6
    # Take the absolute value and replace near-zero values with epsilon
    base = np.abs(x)
    base = np.where(base < epsilon, epsilon, base)
    # Clip the base to avoid extremely large numbers that cause overflow.
    base = np.clip(base, 0, 100)
    return np.power(base, y)

def cube(x):
    """Cube function."""
    return x ** 3

def reciprocal(x):
    """Reciprocal function with protection against division by zero."""
    return 1.0 / (x + 1e-6)

def sigmoid(x):
    """Sigmoid activation function."""
    return 1 / (1 + np.exp(-x))

def gaussian(x):
    """Gaussian function."""
    return np.exp(-x ** 2)

def relu(x):
    """Rectified Linear Unit function."""
    return np.maximum(0, x)

def leaky_relu(x, alpha=0.01):
    """Leaky ReLU function."""
    return np.where(x > 0, x, alpha * x)

def elu(x, alpha=1.0):
    """Exponential Linear Unit function."""
    return np.where(x > 0, x, alpha * (np.exp(x) - 1))

def swish(x):
    """Swish activation function."""
    return x * sigmoid(x)

def mish(x):
    """Mish activation function."""
    return x * np.tanh(np.log1p(np.exp(x)))


def sin1_over_x(x):
    """Sine of 1/x function with protection against division by zero."""
    return np.sin(1.0 / (x + 1e-6))

def sinc(x):
    """Sinc function."""
    return np.sinc(x / np.pi)

def sawtooth(x):
    """Sawtooth wave function."""
    return 2 * (x / (2 * np.pi) - np.floor(0.5 + x / (2 * np.pi)))

def triangle_wave(x):
    """Triangle wave function."""
    return 2 * np.abs(2 * (x / (2 * np.pi) - np.floor(x / (2 * np.pi) + 0.5))) - 1

def square_wave(x):
    """Square wave function."""
    return np.sign(np.sin(x))


def bent_identity(x):
    """Bent identity function."""
    return (np.sqrt(x ** 2 + 1) - 1) / 2 + x

def softsign(x):
    """Softsign function."""
    return x / (1 + np.abs(x))

def hard_sigmoid(x):
    """Hard sigmoid function."""
    return np.clip((x + 1) / 2, 0, 1)

def logit(x):
    """Logit function with protection against division by zero."""
    x = np.clip(x, 1e-6, 1 - 1e-6)
    return np.log(x / (1 - x))


def mod(x, y):
    """Modulo operation with protection against division by zero."""
    return np.mod(x, y + 1e-6)

def max_op(x, y):
    """Maximum of two values."""
    return np.maximum(x, y)

def min_op(x, y):
    """Minimum of two values."""
    return np.minimum(x, y)

def average(x, y):
    """Average of two values."""
    return (x + y) / 2


In [ ]:
"""# -------------------------------
# Load problem data
# -------------------------------
selected_problem = 8
data = np.load(f'../data/problem_{selected_problem}.npz')
x_data = data['x']
y_data = data['y']

MAX_SAMPLES = 2000
if y_data.shape[0] > MAX_SAMPLES:
    step = y_data.shape[0] // MAX_SAMPLES
    y_true = y_data[::step][:MAX_SAMPLES]
    x_data = x_data[:, ::step][:, :MAX_SAMPLES]
else:
    y_true = y_data

# Transpose x_data so that each row is one sample and each column is one variable.
x_data = x_data.T  """


# -------------------------------
# Load problem data
# -------------------------------
selected_problem = 8
data = np.load(f'../data/problem_{selected_problem}.npz')
x_data = data['x']
y_data = data['y']

MAX_SAMPLES = 2000
num_samples = y_data.shape[0]

if num_samples > MAX_SAMPLES:
    # Randomly choose MAX_SAMPLES indices without replacement
    indices = np.random.choice(num_samples, size=MAX_SAMPLES, replace=False)
else:
    indices = np.arange(num_samples)

# Sort indices for consistent plotting (optional)
indices = np.sort(indices)

# Select the same indices from both x and y
y_true = y_data[indices]
x_data = x_data[:, indices]

# Transpose x_data so that each row is one sample and each column is one variable.
x_data = x_data.T


In [44]:
# -------------------------------
# Set up operators and GP parameters
# -------------------------------
# Use one consistent set of operators
operators = [
    operator.add,       # binary: +
    operator.sub,       # binary: -
    operator.mul,       # binary: *
    safe_div,           # binary: protected division
    square,             # unary: square
    cube,               # unary: cube
    custom_sin,         # unary: sine
    custom_cos,         # unary: cosine
    custom_exp,         # unary: exponential (with clipping)
    safe_log,           # unary: protected logarithm
    safe_sqrt,          # unary: protected square root
    custom_tanh,        # unary: tanh
    safe_pow,           # binary: protected power
    reciprocal,         # unary: reciprocal
    sigmoid,            # unary: sigmoid
    gaussian,           # unary: gaussian
    relu,               # unary: ReLU
    leaky_relu,         # unary: leaky ReLU
    elu,                # unary: ELU
    swish,              # unary: swish
    mish,               # unary: mish
    sin1_over_x,        # unary: sin(1/x)
    sinc,               # unary: sinc
    sawtooth,           # unary: sawtooth wave
    triangle_wave,      # unary: triangle wave
    square_wave,        # unary: square wave
    bent_identity,      # unary: bent identity
    softsign,           # unary: softsign
    hard_sigmoid,       # unary: hard sigmoid
    logit,              # unary: logit
    mod,                # binary: modulo
    max_op,             # binary: max
    min_op,             # binary: min
    average             # binary: average
]




# Adjusted parameter settings to allow for more growth and complexity
POP_SIZE = 2000          
NUM_GENS = 100         
OFFSPRING_NUM = 1500      
INITIAL_SIZE_MIN = 1    # Minimum size for initial trees
INITIAL_SIZE_MAX = 25   # Maximum size for initial trees 
TOURN_SIZE = 250        
LENGTH_PENALTY = 0.00001 
CROSSOVER_PROB = 0.7    
MUTATION_PROB = 0.3     
MAX_TREE_DEPTH = 10     # Maximum depth for any tree to prevent bloat
ADAPTIVE_PENALTY = True # Enable adaptive complexity penalty
MIN_SEMANTIC_THRESHOLD = 0.05 






# -------------------------------
# Initialize the GP system with consistent operators
# -------------------------------
# Note: Ensure that 'operators' is defined in your context.



gp = DagGP(
    operators=operators,
    variables=x_data.shape[1],
    constants=np.linspace(-5, 5, 2000)
)







In [45]:
# -------------------------------
# Tree Structure Functions
# -------------------------------

def has_cycle(node, visited=None):
    """Check if a tree has cycles, which should be avoided in GP."""
    if visited is None:
        visited = set()
    if id(node) in visited:
        return True
    visited.add(id(node))
    for child in node.successors:
        if has_cycle(child, visited.copy()):
            return True
    return False


def validate_and_fix_tree(root):
    """If a tree has cycles, replace it with a valid one."""
    if has_cycle(root):
        new_size = np.random.randint(INITIAL_SIZE_MIN, INITIAL_SIZE_MAX + 1)
        return gp.create_individual(new_size)
    return root


def collect_nodes(node, nodes_list=None):
    """Collect all nodes in a tree into a list."""
    if nodes_list is None:
        nodes_list = []
    nodes_list.append(node)
    for child in node.successors:
        collect_nodes(child, nodes_list)
    return nodes_list

def get_tree_depth(node, depth=0, max_depth=0):
    """Calculate the maximum depth of a tree."""
    current_depth = depth + 1
    max_depth = max(max_depth, current_depth)
    for child in node.successors:
        max_depth = get_tree_depth(child, current_depth, max_depth)
    return max_depth

In [46]:
# -------------------------------
# Define an improved fitness evaluation function with adaptive complexity penalty
# -------------------------------
def unified_compute_fitness(ind, generation=0, adaptive=True):
    
    ind = validate_and_fix_tree(ind)

    try:
        pred = gp.evaluate(ind, x_data, variable_names=[f'x{i}' for i in range(x_data.shape[1])])
        pred = np.array(pred)

        if np.any(np.isnan(pred)) or np.any(np.isinf(pred)):
            return -1e30  # Extremely bad fitness

        mse_val = np.mean((pred - y_true) ** 2)

        complexity = len(collect_nodes(ind))

        #base_penalty = 0.001
        penalty_weight = LENGTH_PENALTY * max(0.1, 1.0 - generation / NUM_GENS)

        total_penalty = penalty_weight * complexity

        fitness = -(mse_val + total_penalty)
        return fitness

    except Exception as e:
        print(f"Fitness evaluation failed: {e}")
        return -1e30




def semantic_distance(ind1, ind2, x_data):
    y1 = np.array(gp.evaluate(ind1, x_data))
    y2 = np.array(gp.evaluate(ind2, x_data))
    return np.mean(np.abs(y1 - y2))



In [47]:
# -------------------------------
# Helper functions for mutation and crossover
# -------------------------------
def get_mutation_strategies(current_size, growth_factor):
    """Determine mutation strategies and their probabilities based on tree size."""
    if current_size < 15:  # For smaller trees, encourage growth more aggressively
        strategies = [
            ("point_mutation", max(0.1, 0.35 - growth_factor * 0.2)),
            ("subtree_replacement", max(0.1, 0.35 - growth_factor * 0.2)),
            ("grow", min(0.7, 0.2 + growth_factor * 0.4)),  # Increase growth probability
            ("shrink", max(0.05, 0.1 - growth_factor * 0.05))
        ]
    else:  # For larger trees, balanced approach
        strategies = [
            ("point_mutation", 0.25),
            ("subtree_replacement", 0.25),
            ("grow", 0.4),  # Still significant growth probability
            ("shrink", 0.1)
        ]
    
    # Normalize probabilities to ensure they sum to 1.0
    total_prob = sum(prob for _, prob in strategies)
    return [(name, prob/total_prob) for name, prob in strategies]


def select_mutation_strategy(strategies):
    """Select a mutation strategy based on probabilities."""
    strategy_names, probabilities = zip(*strategies)
    return np.random.choice(strategy_names, p=probabilities)


def select_node_for_mutation(all_nodes, mutation_type):
    """Select a node to mutate with bias toward leaf nodes for growth operations."""
    if mutation_type == "grow" and len(all_nodes) > 1:
        # Separate nodes into internal and leaf nodes
        leaf_nodes = [node for node in all_nodes if node.is_leaf]
        if leaf_nodes and np.random.rand() < 0.7:  # 70% chance to select a leaf node for growth
            return np.random.choice(leaf_nodes)
    
    return np.random.choice(all_nodes)

def replace_subtree(node, new_subtree, parent=None, parent_idx=None):

    # If we have direct parent access, we can replace the entire node
    if parent is not None and parent_idx is not None:
        parent_successors = list(parent.successors)
        parent_successors[parent_idx] = new_subtree
        try:
            parent.successors = parent_successors
            return True
        except AssertionError:
            return False
    
    # If node and new_subtree have the same number of successors, we can replace content
    if len(node.successors) == len(new_subtree.successors):
        if hasattr(node, '_op') and hasattr(new_subtree, '_op'):
            node._op = new_subtree._op
        
        if hasattr(node, '_data') and hasattr(new_subtree, '_data'):
            node._data = new_subtree._data
            
        if hasattr(node, 'name') and hasattr(new_subtree, 'name'):
            node.name = new_subtree.name
        
        for i, successor in enumerate(new_subtree.successors):
            node_successors = list(node.successors)
            node_successors[i] = successor
            node.successors = node_successors
            
        return True
    
    return False

def find_parent_and_index(root, node, path=None):
    if path is None:
        path = []
    
    # Check if any of root's successors is the target node
    for i, child in enumerate(root.successors):
        if child is node:
            return root, i
    
    for i, child in enumerate(root.successors):
        result = find_parent_and_index(child, node, path + [(root, i)])
        if result is not None:
            return result
    
    return None

def apply_point_mutation(ind, node, gp_instance, current_depth, max_depth):
    if not node.is_leaf:
        new_op = np.random.choice(operators)
        # Create a small new subtree if there is room
        if current_depth < max_depth:
            # Generate a subtree with potentially more complex structure
            depth_allowance = max_depth - current_depth + 1
            max_new_depth = min(3, depth_allowance)
            new_subtree = gp_instance.create_individual(np.random.randint(2, max_new_depth + 1))
            replace_subtree(node, new_subtree)
    return ind


def apply_subtree_replacement(ind, node, gp_instance, generation, current_depth, max_depth):
    if current_depth >= max_depth:
        return ind
        
    parent_info = find_parent_and_index(ind, node)
    
    # Create a new subtree
    base_size = 3 + int((generation / NUM_GENS) * 5)
    size_range = max(2, base_size + np.random.randint(0, 3))
    new_size = np.random.randint(2, size_range + 1)
    
    # Try up to 5 times to create a compatible subtree
    for _ in range(5):
        new_subtree = gp_instance.create_individual(new_size)
        
        # If we have parent info, try direct replacement
        if parent_info:
            parent, idx = parent_info
            parent_successors = list(parent.successors)
            parent_successors[idx] = new_subtree
            try:
                parent.successors = parent_successors
                return ind
            except AssertionError:
                continue  # Try again with a new subtree
        
        # If node and new_subtree have same arity, try content replacement
        if len(node.successors) == len(new_subtree.successors):
            success = replace_subtree(node, new_subtree)
            if success:
                return ind
    
    # If all attempts failed, return unchanged individual
    return ind

def apply_grow_mutation(ind, node, gp_instance, generation, current_depth, max_depth):
    if current_depth >= max_depth:
        return ind
        
    # If node is a leaf, find its parent to replace it
    if node.is_leaf:
        parent_info = find_parent_and_index(ind, node)
        if parent_info:
            parent, idx = parent_info
            
            # Create a new subtree
            max_growth = max(3, int(2 + (generation / NUM_GENS) * 5))
            new_size = np.random.randint(2, max_growth + 1)
            
            # Try up to 5 times to create a compatible replacement
            for _ in range(5):
                new_subtree = gp_instance.create_individual(new_size)
                parent_successors = list(parent.successors)
                parent_successors[idx] = new_subtree
                try:
                    parent.successors = parent_successors
                    return ind
                except AssertionError:
                    continue  # Try again
    else:
        # For non-leaf nodes, try to replace one of its children with a larger subtree
        if node.successors:
            child_idx = np.random.randint(0, len(node.successors))
            child = node.successors[child_idx]
            
            # Create a new subtree
            max_growth = max(3, int(2 + (generation / NUM_GENS) * 5))
            new_size = np.random.randint(2, max_growth + 1)
            
            # Try up to 5 times to create a compatible replacement
            for _ in range(5):
                new_subtree = gp_instance.create_individual(new_size)
                node_successors = list(node.successors)
                node_successors[child_idx] = new_subtree
                try:
                    node.successors = node_successors
                    return ind
                except AssertionError:
                    continue  # Try again
    
    return ind


def apply_shrink_mutation(ind, node):
    if not node.is_leaf and node.successors:
        # Find parent for the node to mutate
        parent_nodes = [node for node in collect_nodes(ind) if node in node.successors]
        if parent_nodes:
            parent = parent_nodes[0]
            parent_successors = parent.successors[:]
            child_idx = parent_successors.index(node)
            
            # Replace with one of node's children
            if node.successors:
                replacement = np.random.choice(node.successors)
                parent_successors[child_idx] = replacement
                parent.successors = parent_successors
    return ind

def select_crossover_point(nodes, growth_bias):
    # Skip root node
    if len(nodes) <= 1:
        return None
    
    # Separate internal nodes (with children) from leaf nodes
    internal_nodes = [n for n in nodes[1:] if not n.is_leaf]
    
    # If we have internal nodes, use them with higher probability based on growth bias
    if internal_nodes and np.random.rand() < growth_bias:
        return np.random.choice(internal_nodes)
    else:
        return np.random.choice(nodes[1:])  # Skip root

def swap_subtrees(parent1, parent2, point1, point2):
    # Get index of crossover points
    idx1 = parent1.successors.index(point1)
    idx2 = parent2.successors.index(point2)
    
    # Swap subtrees
    successors1 = parent1.successors[:]
    successors2 = parent2.successors[:]
    successors1[idx1], successors2[idx2] = point2, point1
    parent1.successors = successors1
    parent2.successors = successors2

def is_valid_depth(child1, child2, generation):
    depth1 = get_tree_depth(child1)
    depth2 = get_tree_depth(child2)
    
    # More permissive depth check that relaxes as generations progress
    max_allowed_depth = MAX_TREE_DEPTH
    if np.random.rand() < (generation / NUM_GENS) * 0.3:
        # Occasionally allow slightly deeper trees in later generations
        max_allowed_depth += 2
        
    return depth1 <= max_allowed_depth and depth2 <= max_allowed_depth

In [48]:
# -------------------------------
# Streamlined Selection, Mutation, and Crossover Operations
# -------------------------------
def tournament_select(pop, generation=0, size_aware=False):
    
    participants = np.random.choice(pop, size=min(TOURN_SIZE, len(pop)), replace=False)
    
    if size_aware and np.random.rand() < min(0.3, (generation / NUM_GENS) * 0.4):
        avg_size = np.mean([len(collect_nodes(ind)) for ind in pop])
        
        # Occasionally favor larger trees if they're above average size
        # but not too far below average fitness
        sorted_participants = sorted(participants, key=lambda ind: ind.fitness, reverse=True)
        
        for ind in sorted_participants:
            ind_size = len(collect_nodes(ind))
            if ind_size > avg_size * 1.2:  # Tree is 20% larger than average
                # If fitness is at least 80% of the best participant, select it
                best_fitness = sorted_participants[0].fitness
                if ind.fitness >= best_fitness * 0.8:
                    return ind
    
    # Default to regular tournament selection
    return max(participants, key=lambda ind: ind.fitness)




def enhanced_mutate(ind, gp_instance, generation):
    
    ind_copy = copy.deepcopy(ind)
    
    current_size = len(collect_nodes(ind_copy))
    current_depth = get_tree_depth(ind_copy)
    
    # Growth factor increases with generation
    growth_factor = min(0.8, generation / NUM_GENS)
    
    strategies = get_mutation_strategies(current_size, growth_factor)
    
    strategy = select_mutation_strategy(strategies)
    
    all_nodes = collect_nodes(ind_copy)
    
    # Select a node to mutate, with bias toward appropriate nodes for the strategy
    node = select_node_for_mutation(all_nodes, strategy)
    
    if strategy == "point_mutation":
        ind_copy = apply_point_mutation(ind_copy, node, gp_instance, current_depth, MAX_TREE_DEPTH)
    elif strategy == "subtree_replacement":
        ind_copy = apply_subtree_replacement(ind_copy, node, gp_instance, generation, current_depth, MAX_TREE_DEPTH)
    elif strategy == "grow":
        ind_copy = apply_grow_mutation(ind_copy, node, gp_instance, generation, current_depth, MAX_TREE_DEPTH)
    elif strategy == "shrink":
        ind_copy = apply_shrink_mutation(ind_copy, node)
    
    # Validate the tree after mutation
    ind_copy = validate_and_fix_tree(ind_copy)
    
    return ind_copy


def growth_enhanced_crossover(parent1, parent2, generation):
    
    # Make deep copies to avoid modifying the parents
    child1 = copy.deepcopy(parent1)
    child2 = copy.deepcopy(parent2)
    
    # Allow crossover to become more aggressive in later generations
    growth_bias = min(0.8, 0.3 + (generation / NUM_GENS) * 0.5)
    
    # Select nodes from both parents, with increasing bias towards internal nodes
    # that could lead to more interesting genetic material exchange
    nodes1 = collect_nodes(child1)
    nodes2 = collect_nodes(child2)
    
    point1 = select_crossover_point(nodes1, growth_bias)
    point2 = select_crossover_point(nodes2, growth_bias)
    
    # If valid crossover points found in both trees
    if point1 and point2:
        # Find parent nodes for each selected crossover point
        parent_nodes1 = [n for n in nodes1 if point1 in n.successors]
        parent_nodes2 = [n for n in nodes2 if point2 in n.successors]
        
        if parent_nodes1 and parent_nodes2:
            # Perform the crossover by swapping subtrees
            swap_subtrees(parent_nodes1[0], parent_nodes2[0], point1, point2)
            
            if not is_valid_depth(child1, child2, generation):
                # If constraints violated, create new children with appropriate sizes
                size1 = len(collect_nodes(parent1))
                size2 = len(collect_nodes(parent2))
                child1 = gp.create_individual(size1)
                child2 = gp.create_individual(size2)
    
    return child1, child2



In [49]:
# Diversity injection that creates larger trees in later generations
def enhanced_diversity_injection(population, gp_instance, generation):
    diversity = calculate_population_diversity(population)
    
    # Calculate average tree size
    sizes = [len(collect_nodes(ind)) for ind in population]
    avg_size = np.mean(sizes)
    max_size = max(sizes)
    
    # More aggressive diversity injection as generations progress
    diversity_threshold = max(0.1, 0.3 - (generation / NUM_GENS) * 0.1)
    
    # Inject diversity if population is too homogeneous or trees are too small
    if diversity < diversity_threshold or (generation > NUM_GENS * 0.5 and max_size < 20):
        print(f"Generation {generation}: Low diversity ({diversity:.2f}) or small trees (avg:{avg_size:.1f}, max:{max_size}). Injecting new individuals.")
        
        # Replace more individuals in later generations
        replacement_rate = min(0.2, 0.1 + (generation / NUM_GENS) * 0.1)
        num_to_replace = max(1, int(replacement_rate * len(population)))
        
        if generation > NUM_GENS * 0.4 and avg_size < 15:
            # Replace smallest trees with some probability
            candidates = [(i, ind) for i, ind in enumerate(population) 
                         if len(collect_nodes(ind)) < avg_size * 0.8]
            if candidates:
                # Replace some smallest trees
                small_indices = [i for i, _ in sorted(candidates, key=lambda x: len(collect_nodes(x[1])))[:num_to_replace//2]]
                # replace some lowest fitness individuals
                sorted_pop = sorted(enumerate(population), key=lambda x: x[1].fitness)
                low_fitness_indices = [i for i, _ in sorted_pop[:num_to_replace - len(small_indices)]]
                indices_to_replace = small_indices + low_fitness_indices
            else:
                # Default to replacing lowest fitness
                sorted_pop = sorted(enumerate(population), key=lambda x: x[1].fitness)
                indices_to_replace = [i for i, _ in sorted_pop[:num_to_replace]]
        else:
            # Default to replacing lowest fitness
            sorted_pop = sorted(enumerate(population), key=lambda x: x[1].fitness)
            indices_to_replace = [i for i, _ in sorted_pop[:num_to_replace]]
        
        # Create new individuals with size scaling with generation
        new_individuals = []
        for _ in range(len(indices_to_replace)):
            # Progressively larger trees as generations increase
            min_size = INITIAL_SIZE_MIN + int((generation / NUM_GENS) * 3)
            max_size = INITIAL_SIZE_MAX + int((generation / NUM_GENS) * 15)
            
            size = np.random.randint(min_size, max_size + 1)
            new_ind = gp_instance.create_individual(size)
            new_ind.fitness = unified_compute_fitness(new_ind, generation)
            new_individuals.append(new_ind)
        
        # Replace selected individuals
        for idx, new_ind in zip(indices_to_replace, new_individuals):
            population[idx] = new_ind
    
    return population

# -------------------------------
# Adaptive Diversity Maintenance
# -------------------------------
def calculate_population_diversity(population, sample_size=None):
    
    if sample_size and sample_size < len(population):
        individuals = random.sample(population, sample_size)
    else:
        individuals = population
    
    n = len(individuals)
    if n <= 1:
        return 0.0
    
    # Calculate pairwise distances
    total_distance = 0.0
    pair_count = 0
    
    for i in range(n):
        for j in range(i+1, n):
            distance = semantic_distance(individuals[i], individuals[j], x_data)
            if distance != float('inf'):
                total_distance += distance
                pair_count += 1
                
    # Normalize and return diversity score
    if pair_count > 0:
        avg_distance = total_distance / pair_count
        normalized_diversity = min(1.0, avg_distance / 10.0)
        return normalized_diversity
    else:
        return 0.0

def enhanced_diversity_injection(population, gp_instance, generation):
    
    # Early generations: subtle diversity, later generations: more aggressive
    diversity_rate = 0.05 + (generation / NUM_GENS) * 0.15
    
    current_diversity = calculate_population_diversity(population, sample_size=min(20, len(population)))
    
    # Only inject if diversity is low
    if current_diversity < 0.3 or (generation % 10 == 0 and generation > 0):
        num_to_inject = max(1, int(POP_SIZE * diversity_rate))
        
        # Identify individuals to replace (lower fitness ones)
        population_sorted = sorted(population, key=lambda ind: ind.fitness)
        to_replace = population_sorted[:num_to_inject]
        
        # Create new diverse individuals
        for i in range(num_to_inject):
            # More aggressive size scaling with generation
            base_size = 3 + int((generation / NUM_GENS) * 12)
            size_range = max(INITIAL_SIZE_MIN, base_size + np.random.randint(-2, 5))
            size = np.random.randint(INITIAL_SIZE_MIN, size_range + 1)
            
            new_ind = gp_instance.create_individual(size)
            new_ind.fitness = unified_compute_fitness(new_ind, generation)
            
            # Replace in the population
            idx = population.index(to_replace[i])
            population[idx] = new_ind
            
        print(f"Generation {generation}: Injected {num_to_inject} new individuals for diversity")
    
    return population

In [50]:
# -------------------------------
# Initialize population with more diversity
# -------------------------------


#USELESS, REMOVED FROM MAIN

# Try to include specialized solutions like sin(x0)
#used done
def try_create_special_solutions():
    """Try to create potential solutions directly."""
    special_solutions = []
    
    # Make sure we're using the correct variable naming convention
    # Check how many variables we have first
    num_vars = x_data.shape[1]
    
    # Create terminal nodes by directly instantiating Node with a string,
    # which, per the teacher library, creates a terminal.
    if num_vars >= 1:
        try:
            # Create sin(x0)
            x0_node = Node("x0")
            sin_node = Node(custom_sin, [x0_node], name="custom_sin")
            special_solutions.append(sin_node)
        except Exception as e:
            print(f"Error creating sin(x0): {e}")
    
    if num_vars >= 2:
        try:
            # Create x0 + x1 using the library's Node constructor for operators
            x0_node = Node("x0")
            x1_node = Node("x1")
            add_node = Node(operator.add, [x0_node, x1_node], name="add")
            special_solutions.append(add_node)
        except Exception as e:
            print(f"Error creating x0 + x1: {e}")
        
        try:
            # Create x0 * x1
            x0_node = Node("x0")
            x1_node = Node("x1")
            mul_node = Node(operator.mul, [x0_node, x1_node], name="mul")
            special_solutions.append(mul_node)
        except Exception as e:
            print(f"Error creating x0 * x1: {e}")
    
    if num_vars >= 1:
        try:
            # Create sin(cos(x0))
            x0_node = Node("x0")
            cos_node = Node(custom_cos, [x0_node], name="custom_cos")
            sin_node = Node(custom_sin, [cos_node], name="custom_sin")
            special_solutions.append(sin_node)
        except Exception as e:
            print(f"Error creating sin(cos(x0)): {e}")
    
    return special_solutions





# -------------------------------

#do i really need this? the main is below


"""# Add special solutions to the population
valid_solutions = []
special_solutions = try_create_special_solutions()

for solution in special_solutions:
    try:
        # Test if the solution can be evaluated
        _ = gp.evaluate(solution, x_data[:1], variable_names=[f'x{i}' for i in range(x_data.shape[1])])
        solution.fitness = unified_compute_fitness(solution)
        valid_solutions.append(solution)
    except Exception as e:
        print(f"Error evaluating solution: {e}")

# Now add the valid solutions to the population
for solution in valid_solutions:
    if len(population) >= POP_SIZE:
        # Replace a random individual with lower fitness.
        lower_half = sorted(population, key=lambda ind: ind.fitness)[:POP_SIZE // 2]
        to_replace = np.random.choice(lower_half)
        population.remove(to_replace)
    
    population.append(solution)"""

'# Add special solutions to the population\nvalid_solutions = []\nspecial_solutions = try_create_special_solutions()\n\nfor solution in special_solutions:\n    try:\n        # Test if the solution can be evaluated\n        _ = gp.evaluate(solution, x_data[:1], variable_names=[f\'x{i}\' for i in range(x_data.shape[1])])\n        solution.fitness = unified_compute_fitness(solution)\n        valid_solutions.append(solution)\n    except Exception as e:\n        print(f"Error evaluating solution: {e}")\n\n# Now add the valid solutions to the population\nfor solution in valid_solutions:\n    if len(population) >= POP_SIZE:\n        # Replace a random individual with lower fitness.\n        lower_half = sorted(population, key=lambda ind: ind.fitness)[:POP_SIZE // 2]\n        to_replace = np.random.choice(lower_half)\n        population.remove(to_replace)\n    \n    population.append(solution)'

In [51]:



def run_streamlined_evolution():
    
    global LENGTH_PENALTY, population
    
    history = {
        'best_fitness': [],
        'avg_fitness': [],
        'complexity': [],
        'diversity': []
    }
    
    state = {
        'stagnation_counter': 0,
        'last_best_fitness': float('-inf'),
        'growth_stagnation': 0,
        'max_size_seen': max(len(collect_nodes(ind)) for ind in population),
        'found_special_solution': False,
        'special_solution_gen': -1,
        'best_formula_found': ""
    }
    
    # Sort initial population by fitness (highest first)
    population = sorted(population, key=lambda ind: ind.fitness, reverse=True)
    
    # Main evolutionary loop
    for gen in tqdm(range(NUM_GENS), desc="Generations", leave=True):
        # ---- PHASE 1: Diversity and Offspring Generation ----
        
        # Inject diversity to prevent premature convergence
        population = enhanced_diversity_injection(population, gp, gen)
        
        # Determine offspring count based on stagnation indicators
        offspring_size = OFFSPRING_NUM
        if state['stagnation_counter'] > 10:
            offspring_size = int(OFFSPRING_NUM * 1.5)
            tqdm.write(f"Generation {gen}: Fitness stagnation detected. Increasing offspring to {offspring_size}")
        if state['growth_stagnation'] > 8:
            offspring_size = int(offspring_size * 1.2)
            tqdm.write(f"Generation {gen}: Size stagnation detected. Further increasing offspring to {offspring_size}")
        
        # ---- PHASE 2: Generate New Offspring ----
        new_offspring = []
        
        # Generate offspring through crossover and mutation
        while len(new_offspring) < offspring_size:
            # Select first parent using tournament selection
            parent1 = tournament_select(population, gen)                    
            
            if np.random.rand() < CROSSOVER_PROB:
                # Crossover operation
                parent2 = tournament_select(population, gen)
                child1, child2 = growth_enhanced_crossover(parent1, parent2, gen)
                
                # Validate children
                child1 = validate_and_fix_tree(child1)
                child2 = validate_and_fix_tree(child2)
                
                # Apply semantic check and additional mutation if needed
                if semantic_distance(child1, parent1, x_data) < MIN_SEMANTIC_THRESHOLD:
                    child1 = enhanced_mutate(child1, gp, gen)
                    child1 = validate_and_fix_tree(child1)
                    
                new_offspring.append(child1)
                
                # Add second child if space allows
                if len(new_offspring) < offspring_size:
                    if semantic_distance(child2, parent2, x_data) < MIN_SEMANTIC_THRESHOLD:
                        child2 = enhanced_mutate(child2, gp, gen)
                        child2 = validate_and_fix_tree(child2)
                    new_offspring.append(child2)
            else:
                # Mutation operation
                mutant = enhanced_mutate(parent1, gp, gen)
                mutant = validate_and_fix_tree(mutant)
                
                # Check semantic difference and possibly mutate again
                if semantic_distance(mutant, parent1, x_data) < MIN_SEMANTIC_THRESHOLD:
                    mutant = enhanced_mutate(mutant, gp, gen)
                    mutant = validate_and_fix_tree(mutant)
                    
                new_offspring.append(mutant)
                
            # Stop after generating enough offspring
            if len(new_offspring) >= offspring_size:
                break
        
        # ---- PHASE 3: Evaluate Offspring ----
        for ind in new_offspring:
            ind.fitness = unified_compute_fitness(ind, gen)
        
        # ---- PHASE 4: Selection with Size Diversity Preservation ----
        population.extend(new_offspring)
        
        # Selection strategy
        if gen > NUM_GENS * 0.3 and np.random.rand() < 0.3:
            # Occasionally preserve some large trees to maintain structural diversity
            population.sort(key=lambda ind: ind.fitness, reverse=True)
            selected = population[:int(POP_SIZE * 0.9)]
            remaining = population[int(POP_SIZE * 0.9):]
            largest_trees = sorted(remaining, key=lambda ind: len(collect_nodes(ind)), reverse=True)
            selected.extend(largest_trees[:POP_SIZE - len(selected)])
            population = selected
        else:
            # Standard selection by fitness
            population = sorted(population, key=lambda ind: ind.fitness, reverse=True)[:POP_SIZE]
        
        # ---- PHASE 5: Compute Statistics ----
        best_fitness = population[0].fitness
        avg_fitness = np.mean([ind.fitness for ind in population])
        best_complexity = len(collect_nodes(population[0]))
        current_max_size = max(len(collect_nodes(ind)) for ind in population)
        diversity = calculate_population_diversity(population, sample_size=min(20, len(population)))
        
        # Update history
        history['best_fitness'].append(best_fitness)
        history['avg_fitness'].append(avg_fitness)
        history['complexity'].append(best_complexity)
        history['diversity'].append(diversity)
        
        # ---- PHASE 6: Stagnation Detection and Response ----
        
        # Check fitness stagnation
        if best_fitness <= state['last_best_fitness'] + 1e-6:
            state['stagnation_counter'] += 1
        else:
            state['stagnation_counter'] = 0
            state['last_best_fitness'] = best_fitness
        
        # Check growth stagnation
        if current_max_size <= state['max_size_seen']:
            state['growth_stagnation'] += 1
        else:
            state['growth_stagnation'] = 0
            state['max_size_seen'] = current_max_size
        
        # ---- PHASE 7: Adaptive Control Mechanisms ----
        
        # Control size explosion if trees are growing too fast without fitness improvement
        if (current_max_size > 50 and state['stagnation_counter'] > 15) or current_max_size > 100:
            old_penalty = LENGTH_PENALTY
            LENGTH_PENALTY = old_penalty * 2
            tqdm.write(f"Generation {gen}: Controlling bloat. Increasing penalty temporarily from {old_penalty:.6f} to {LENGTH_PENALTY:.6f}")
            for ind in population:
                ind.fitness = unified_compute_fitness(ind, gen)
            if gen > 3:
                LENGTH_PENALTY = old_penalty
                
        # Accelerate growth when trees are too small
        if gen > 10 and state['max_size_seen'] < 15 and state['growth_stagnation'] > 5:
            old_penalty = LENGTH_PENALTY
            LENGTH_PENALTY = old_penalty * 0.1
            tqdm.write(f"Generation {gen}: Trees too small. Drastically reducing complexity penalty from {old_penalty:.6f} to {LENGTH_PENALTY:.6f}")
            
            # Inject larger trees into population
            num_large_trees = POP_SIZE // 10
            for _ in range(num_large_trees):
                size = np.random.randint(15, 25)
                new_ind = gp.create_individual(size)
                new_ind.fitness = unified_compute_fitness(new_ind, gen)
                
                # Replace individuals from the bottom half of the population
                bottom_half = sorted(population, key=lambda ind: ind.fitness)[:POP_SIZE // 2]
                to_replace = np.random.choice(bottom_half)
                idx = population.index(to_replace)
                population[idx] = new_ind
                
            state['growth_stagnation'] = 0
        
        # ---- PHASE 8: Special Solution Detection ----
        #used at the beginning of the project, in the problem 1 to see if i got the right solution
        best_formula = population[0].long_name.lower()
        
        """if not state['found_special_solution']:
            if "sin(x0)" in best_formula or "custom_sin(x0)" in best_formula:
                state['found_special_solution'] = True
                state['special_solution_gen'] = gen
                state['best_formula_found'] = "sin(x0)"
                tqdm.write(f"Found sin(x0) at generation {gen}!")"""
        
        # ---- PHASE 9: Logging ----
        if gen % 5 == 0 or gen == NUM_GENS - 1:
            current_sizes = [len(collect_nodes(ind)) for ind in population]
            tqdm.write(f"Generation {gen}: Best Fitness = {best_fitness:.6f}, Avg Fitness = {avg_fitness:.6f}")
            tqdm.write(f"Tree sizes: Min = {min(current_sizes)}, Avg = {sum(current_sizes)/len(current_sizes):.1f}, Max = {max(current_sizes)}")
            tqdm.write(f"Best complexity: {best_complexity}, Diversity: {diversity:.2f}")
            tqdm.write(f"Best formula: {population[0].long_name}")
    
    return (population, history['best_fitness'], history['avg_fitness'], 
            history['complexity'], history['diversity'], state['found_special_solution'], 
            state['special_solution_gen'], state['best_formula_found'])

In [52]:
def main():
    
    # Set random seed for reproducibility
    np.random.seed(42)
    
    print(f"Loading problem {selected_problem}...")
    print(f"Data shape: x={x_data.shape}, y={y_true.shape}")
    
    # Initialize operators and GP system
    print("Initializing GP system...")
    global gp, population
    
    
    # Initialize population with diverse individuals
    print("Initializing population...")
    population = []
    
    # Initialize with diverse sizes
    for _ in range(POP_SIZE):
        size = np.random.randint(INITIAL_SIZE_MIN, INITIAL_SIZE_MAX + 1)
        ind = gp.create_individual(size)
        ind.fitness = unified_compute_fitness(ind)
        population.append(ind)
    
    
    
    print(f"Initial population size: {len(population)}")
    
    # Run the evolutionary algorithm
    print("\nStarting evolutionary process...")
    (final_population, best_fitness_history, avg_fitness_history, 
     complexity_history, diversity_history, found_special_solution, 
     special_solution_gen, best_formula_found) = run_streamlined_evolution()    #-------------------------------------------------------------
    
    
    print("\nEvolution completed!")
    
    # Plot fitness evolution and population statistics
    print("Generating plots...")
    plt.figure(figsize=(15, 12))
    
    # Plot fitness evolution
    plt.subplot(3, 1, 1)
    plt.plot(best_fitness_history, label="Best Fitness")
    #plt.plot(avg_fitness_history, label="Average Fitness")
    if found_special_solution:
        plt.axvline(x=special_solution_gen, color='r', linestyle='--', 
                    label=f"{best_formula_found} found (gen {special_solution_gen})")
    plt.xlabel("Generation")
    plt.ylabel("Fitness (negative MSE with penalty)")
    plt.title("Evolution of Fitness")
    plt.legend()
    plt.grid(True)
    
    # Plot complexity
    plt.subplot(3, 1, 2)
    plt.plot(complexity_history, label="Complexity (nodes)")
    plt.xlabel("Generation")
    plt.ylabel("Number of Nodes")
    plt.title("Complexity of Best Individual")
    plt.legend()
    plt.grid(True)
    
    # Plot diversity
    plt.subplot(3, 1, 3)
    plt.plot(diversity_history, label="Population Diversity", color="green")
    plt.xlabel("Generation")
    plt.ylabel("Diversity (semantic distance)")
    plt.title("Population Diversity")
    plt.legend()
    plt.grid(True)
    
    plt.tight_layout()
    plt.savefig(f"evolution_problem_{selected_problem}.png")
    plt.show()
    
    # Visualize and Print the Best Individual
    best_individual = final_population[0]
    try:
        best_individual.draw()  # Leverage the teacher library's built-in draw method
        plt.title("Best GP Expression Tree")
        plt.savefig(f"best_tree_problem_{selected_problem}.png")
        plt.show()
    except Exception as e:
        print(f"Tree visualization failed: {e}")
    
    print("\nBest formula:", best_individual.long_name)
    
    # Compare Model Prediction vs Ground Truth
    predicted = gp.evaluate(best_individual, x_data, 
                            variable_names=[f'x{i}' for i in range(x_data.shape[1])])
    plt.figure(figsize=(12, 6))
    plt.plot(predicted, label="GP Prediction", color="blue")
    plt.plot(y_true, label="True Data", color="red", linestyle="--")
    plt.xlabel("Data Index")
    plt.ylabel("Output")
    plt.title("GP Model Prediction vs Ground Truth")
    plt.legend()
    plt.grid(True)
    plt.savefig(f"prediction_problem_{selected_problem}.png")
    plt.show()
    
    # Calculate MSE to evaluate solution quality
    mse = np.mean((np.array(predicted) - y_true) ** 2)
    print(f"\nMean Squared Error: {mse:.6f}")
    
    # Split data into training and validation sets
    split_index = int(0.8 * len(y_true))
    x_train = x_data[:split_index]
    x_validation = x_data[split_index:]
    y_train = y_true[:split_index]
    y_validation = y_true[split_index:]
    
    def evaluate_best(x):
        return gp.evaluate(best_individual, x, variable_names=[f'x{i}' for i in range(x.shape[1])])
    
    pred_train = evaluate_best(x_train)
    pred_validation = evaluate_best(x_validation)
    
    # Print MSE using teacher's style
    train_mse = 100 * np.square(np.array(y_train) - np.array(pred_train)).sum() / len(y_train)
    validation_mse = 100 * np.square(np.array(y_validation) - np.array(pred_validation)).sum() / len(y_validation)
    
    print(f"MSE (train): {train_mse:g}")
    print(f"MSE (validation): {validation_mse:g}")
    
    # Save best solution information
    with open(f"solution_problem_{selected_problem}.txt", "w") as f:
        f.write(f"Best formula: {best_individual.long_name}\n")
        f.write(f"MSE (train): {train_mse:g}\n")
        f.write(f"MSE (validation): {validation_mse:g}\n")
        f.write(f"Complexity (nodes): {len(collect_nodes(best_individual))}\n")
        f.write(f"Final fitness: {best_individual.fitness}\n")
        if found_special_solution:
            f.write(f"Found {best_formula_found} at generation {special_solution_gen}\n")
    
    print("\nResults saved to files.")
    return best_individual

if __name__ == "__main__":
    best_solution = main()

Loading problem 8...
Data shape: x=(2000, 6), y=(2000,)
Initializing GP system...
Initializing population...


C:\Users\matti\AppData\Local\Temp\ipykernel_14616\1686486328.py:53: RuntimeWarning: overflow encountered in power
  return np.power(base, y)
C:\Users\matti\AppData\Local\Temp\ipykernel_14616\1686486328.py:81: RuntimeWarning: overflow encountered in exp
  return np.where(x > 0, x, alpha * (np.exp(x) - 1))
C:\Users\matti\AppData\Local\Temp\ipykernel_14616\1686486328.py:65: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(-x))
C:\Users\matti\AppData\Local\Temp\ipykernel_14616\1686486328.py:81: RuntimeWarning: invalid value encountered in scalar multiply
  return np.where(x > 0, x, alpha * (np.exp(x) - 1))
C:\Users\matti\AppData\Local\Temp\ipykernel_14616\1120364458.py:15: RuntimeWarning: overflow encountered in square
  mse_val = np.mean((pred - y_true) ** 2)
C:\Users\matti\AppData\Local\Temp\ipykernel_14616\1686486328.py:106: RuntimeWarning: invalid value encountered in scalar subtract
  return 2 * np.abs(2 * (x / (2 * np.pi) - np.floor(x / (2 * np.pi) + 0.5))) - 1
C:

Initial population size: 2000

Starting evolutionary process...


Generations:   0%|          | 0/100 [00:00<?, ?it/s]

C:\Users\matti\AppData\Local\Temp\ipykernel_14616\1686486328.py:115: RuntimeWarning: overflow encountered in scalar power
  return (np.sqrt(x ** 2 + 1) - 1) / 2 + x


...

C:\Users\matti\AppData\Local\Temp\ipykernel_14616\1686486328.py:77: RuntimeWarning: overflow encountered in scalar multiply
  return np.where(x > 0, x, alpha * x)
C:\Users\matti\AppData\Local\Temp\ipykernel_14616\1686486328.py:77: RuntimeWarning: invalid value encountered in multiply
  return np.where(x > 0, x, alpha * x)
c:\Users\matti\OneDrive\Desktop\Uni\Quinto_anno\Computational_intelligence\Labs\Squillero\computational-intelligence\2024-25\project-work\src\gxgp\node.py:27: RuntimeWarning: overflow encountered in scalar multiply
  return node(*_args)


Generation 15: Best Fitness = -10470614.259777, Avg Fitness = -10472920.162238
Tree sizes: Min = 39, Avg = 48.9, Max = 71
Best complexity: 69, Diversity: 0.15
Best formula: safe_div(mod(custom_tanh(elu(safe_pow(add(0.567784, add(0.507754, x4)), -2.90895), x5)), mod(custom_tanh(elu(safe_pow(add(0.507754, x4), -2.90895), 0.567784)), mod(custom_tanh(elu(safe_pow(add(0.507754, add(0.507754, x4)), -2.90895), x5)), mod(custom_tanh(elu(safe_pow(add(0.507754, x4), -2.90895), 0.567784)), mod(custom_tanh(safe_pow(add(0.507754, add(0.507754, x4)), -2.90895)), mod(custom_tanh(elu(safe_pow(add(0.507754, x4), -2.90895), 0.567784)), sub(x5, square(mul(-2.32366, elu(4.33467, 0.567784)))))))))), custom_exp(x5))
Generation 16: Injected 148 new individuals for diversity


C:\Users\matti\AppData\Local\Temp\ipykernel_14616\1686486328.py:24: RuntimeWarning: overflow encountered in square
  return x ** 2
C:\Users\matti\AppData\Local\Temp\ipykernel_14616\1686486328.py:115: RuntimeWarning: invalid value encountered in add
  return (np.sqrt(x ** 2 + 1) - 1) / 2 + x
C:\Users\matti\AppData\Local\Temp\ipykernel_14616\1686486328.py:115: RuntimeWarning: overflow encountered in square
  return (np.sqrt(x ** 2 + 1) - 1) / 2 + x
C:\Users\matti\AppData\Local\Temp\ipykernel_14616\1686486328.py:57: RuntimeWarning: overflow encountered in power
  return x ** 3
C:\Users\matti\AppData\Local\Temp\ipykernel_14616\1686486328.py:85: RuntimeWarning: invalid value encountered in scalar multiply
  return x * sigmoid(x)
C:\Users\matti\AppData\Local\Temp\ipykernel_14616\1686486328.py:85: RuntimeWarning: invalid value encountered in multiply
  return x * sigmoid(x)
c:\Users\matti\OneDrive\Desktop\Uni\Quinto_anno\Computational_intelligence\Labs\Squillero\computational-intelligence\202

Generation 20: Injected 160 new individuals for diversity
Generation 20: Best Fitness = -10470032.807278, Avg Fitness = -10471236.191552
Tree sizes: Min = 41, Avg = 63.0, Max = 79
Best complexity: 76, Diversity: 0.48
Best formula: safe_div(mod(custom_tanh(elu(safe_pow(add(0.507754, x4), -2.90895), x5)), mod(custom_tanh(elu(safe_pow(add(custom_tanh(safe_pow(add(0.507754, add(0.507754, x4)), -2.90895)), x4), -2.90895), 0.567784)), mod(custom_tanh(elu(safe_pow(add(0.507754, add(0.507754, add(0.507754, x4))), -2.90895), x5)), mod(custom_tanh(elu(safe_pow(add(0.507754, x4), -2.90895), 0.567784)), mod(custom_tanh(safe_pow(add(0.507754, add(0.507754, x4)), -2.90895)), mod(custom_tanh(elu(safe_pow(add(0.507754, x4), -2.90895), 0.567784)), sub(x5, square(mul(-2.32366, elu(4.33467, 0.567784)))))))))), custom_exp(x5))


KeyboardInterrupt: 